# Obtención de la información de la base de datos
Por medio de setencias SQL se obtendrá la información para poder realizar filtros de búsqueda para los potenciales usuarios objetivo.

usuario objetivo: Personas de 65 años o más, que realicen movimientos bancarios en giros relacionados con el sector salud.

In [9]:
# Importar librerias
import pandas as pd
import mysql.connector

In [10]:
mydb = mysql.connector.connect(
    host="localhost",
    port="3307",
    user="root",
    passwd="adminroot",
    database="hackathon_bbva"
)

connector = mydb.cursor()

In [11]:
def obtener_giros_de_interes():
    """
    CON BASE EN EL ANÁLSIS REALIZADO DE LOS ARCHIVOS CSV SE CONCLUYÓ QUE
    LOS GIROS Y SUB GIROS DE INTERÉS SON:

        - GIRO: FARMACIAS

        - SUBGIRO: 'DOCTORES, OTRAS ESPECIALIDADES,(MEDICOS)'
        - SUBGIRO: 'OPTOMETRISTAS, (OFTALMOLOGOS)'
        - SUBGIRO: 'LABORATORIOS DENTALES, (LABORATORIOS MEDICOS)'
        - SUBGIRO: 'MEDICINAS, MEDICINAS DE PATENTE, FARMACOS DIVERSOS'
        - SUBGIRO: 'OPTICAS, LENTES Y ANTEOJOS'
        - SUBGIRO: 'SERVICIOS MEDICOS Y PARAMEDICOS (NO CLASIFICADOS)'

        :return: lista de giros y subgiros de interés
    """

    query = """
        SELECT 
            DISTINCT NB_GIRO, NB_SUBGIRO 
        FROM
            catalogo_giros
        WHERE
            NB_GIRO = 'FARMACIAS'
        OR
            NB_SUBGIRO = 'DOCTORES, OTRAS ESPECIALIDADES,(MEDICOS)'
        OR
            NB_SUBGIRO = 'OPTOMETRISTAS, (OFTALMOLOGOS)'
        OR
            NB_SUBGIRO = 'LABORATORIOS DENTALES, (LABORATORIOS MEDICOS)'
        OR
            NB_SUBGIRO = 'MEDICINAS, MEDICINAS DE PATENTE, FARMACOS DIVERSOS'
        OR
            NB_SUBGIRO = 'OPTICAS, LENTES Y ANTEOJOS'
        OR
            NB_SUBGIRO = 'SERVICIOS MEDICOS Y PARAMEDICOS   NO CLASIFICADOS'

        ORDER BY NB_GIRO ASC;
    """

    response = connector.execute(query)

    giros_de_interes = connector.fetchall()
    giros_de_interes = [giro[1] for giro in giros_de_interes]

    return giros_de_interes

obtener_giros_de_interes()


['DROGUERIAS,',
 'DOCTORES, OTRAS ESPECIALIDADES,(MEDICOS)',
 'OPTOMETRISTAS, (OFTALMOLOGOS)',
 'LABORATORIOS DENTALES, (LABORATORIOS MEDICOS)',
 'MEDICINAS, MEDICINAS DE PATENTE, FARMACOS DIVERSOS',
 'OPTICAS, LENTES Y ANTEOJOS',
 'SERVICIOS MEDICOS Y PARAMEDICOS   NO CLASIFICADOS']

In [12]:
def obtener_publico_objetivo():
    """
    EL PÚBLICO OBJETIVO ES EL DE PERSONAS DE 65 AÑOS O MÁS

        :return: lista de 
    """
    query = """
        SELECT
            *
        FROM
            descripcion_cliente
        WHERE
            EDAD >= 65
    """

    response = connector.execute(query)

    publico_objetivo = connector.fetchall()
    publico_objetivo = [cliente[0:] for cliente in publico_objetivo]

    return publico_objetivo

obtener_publico_objetivo()

[('1VLE6Y9G', '79.0', 'CH', '32390.0', 'M'),
 ('ABK0NHDA', '68.0', 'DF', '13360.0', 'M')]

In [13]:
def obtener_clientes():
    """
    OBTENER TODOS LOS CLIENTES

        :return: lista de 
    """
    query = """
        SELECT
            *
        FROM
            descripcion_cliente
    """

    response = connector.execute(query)

    clientes = connector.fetchall()
    clientes = [cliente[0:] for cliente in clientes]

    return clientes

obtener_clientes()

[('04SEMBV0', '35.0', 'DF', '11870.0', 'F'),
 ('0BVB9RAX', '32.0', 'DF', '6200.0', 'F'),
 ('1VLE6Y9G', '79.0', 'CH', '32390.0', 'M'),
 ('29R7LSKX', '50.0', 'DF', '4600.0', 'M'),
 ('3CE1LE8Z', '64.0', 'DU', '34414.0', 'F'),
 ('54ZFPLCJ', '52.0', 'JA', '44270.0', 'F'),
 ('97KQXY8P', '51.0', 'JA', '44770.0', 'M'),
 ('A2LKFXVK', '33.0', 'JA', '45410.0', 'F'),
 ('ABK0NHDA', '68.0', 'DF', '13360.0', 'M'),
 ('ABLZF7FM', '40.0', 'JA', '45070.0', 'M'),
 ('BT03YGRT', '59.0', 'BS', '23477.0', 'M'),
 ('DWPAFY3Q', '30.0', 'DF', '14439.0', 'M'),
 ('GQUR70YV', '30.0', 'JA', '45580.0', 'M'),
 ('JE59J1CC', '41.0', 'VE', '91477.0', 'F'),
 ('M5JVN30B', '22.0', 'JA', '44760.0', 'M'),
 ('MV0G38ER', '47.0', 'CA', '24190.0', 'F'),
 ('QCYZ6SWA', '21.0', 'DF', '1820.0', 'F'),
 ('R0CB78B7', '22.0', 'EM', '52928.0', 'M'),
 ('RMMTWKSD', '22.0', 'AG', '20344.0', 'M'),
 ('SXA5010S', '45.0', 'JA', '45653.0', 'F'),
 ('Y2W6ZZX6', '27.0', 'EM', '55975.0', 'F'),
 ('ZM0GRD8U', '31.0', 'TM', '87086.0', 'F'),
 ('ZRED9BV0',

In [14]:
def movimientos_objetivo():
    """
    SE OBTENDRÁN LOS MOVIMIENTOS BANCARIOS QUE SERVIRÁN
    COMO FLAGS PARA ALERTAR SOBRE UN POSIBLE NUEVO
    CLIENTE OBJETIVO
    """

    query = """
        SELECT
            dc.NU_CTE_COD AS 'CODIGO CLIENTE',
            dc.EDAD AS 'EDAD CLIENTE',
            dc.CD_ESTADO AS 'CD_ESTADO',
            dc.CD_POSTAL AS 'CD_POSTAL',
            dc.CD_SEXO AS 'CD_SEXO',
            tc.FH_CORTE AS 'MES TRANSACCION',
            tc.FH_OPERACION AS 'DIA TRANSACCION',
            cg.NB_GIRO AS 'GIRO COMERCIAL',
            cg.NB_SUBGIRO AS 'SUBGIRO'
        FROM
            descripcion_cliente AS dc
        RIGHT JOIN
            transacciones_cliente AS tc
        ON
            tc.NU_CTE_COD = dc.NU_CTE_COD
        RIGHT JOIN
            catalogo_giros AS cg
        ON
            tc.NU_AFILIACION = cg.NU_AFILIACION
        WHERE
            dc.EDAD >= 65
        AND
            cg.NB_SUBGIRO = '{data}'
    """

    giros_de_interes = obtener_giros_de_interes()

    objetivos = []
        
    for giro in giros_de_interes:
        connector.execute(query.format(data=giro))
        objetivo = connector.fetchall()

        objetivo = [cliente[0:] for cliente in objetivo]
        objetivos.append(objetivo)

    return objetivos

movimientos_objetivo()

[[('1VLE6Y9G',
   '79.0',
   'CH',
   '32390.0',
   'M',
   '01/07/2022',
   '20/07/2022',
   'FARMACIAS',
   'DROGUERIAS,'),
  ('1VLE6Y9G',
   '79.0',
   'CH',
   '32390.0',
   'M',
   '01/07/2022',
   '20/07/2022',
   'FARMACIAS',
   'DROGUERIAS,'),
  ('ABK0NHDA',
   '68.0',
   'DF',
   '13360.0',
   'M',
   '01/07/2022',
   '21/07/2022',
   'FARMACIAS',
   'DROGUERIAS,'),
  ('ABK0NHDA',
   '68.0',
   'DF',
   '13360.0',
   'M',
   '01/08/2022',
   '17/08/2022',
   'FARMACIAS',
   'DROGUERIAS,'),
  ('ABK0NHDA',
   '68.0',
   'DF',
   '13360.0',
   'M',
   '01/05/2022',
   '02/05/2022',
   'FARMACIAS',
   'DROGUERIAS,'),
  ('ABK0NHDA',
   '68.0',
   'DF',
   '13360.0',
   'M',
   '01/07/2022',
   '20/07/2022',
   'FARMACIAS',
   'DROGUERIAS,'),
  ('ABK0NHDA',
   '68.0',
   'DF',
   '13360.0',
   'M',
   '01/07/2022',
   '28/07/2022',
   'FARMACIAS',
   'DROGUERIAS,'),
  ('ABK0NHDA',
   '68.0',
   'DF',
   '13360.0',
   'M',
   '01/04/2022',
   '12/04/2022',
   'FARMACIAS',
   'DROGUER

In [17]:
def obtener_movimientos():
    """
    OBTENEMOS TODOS LOS MOVIMIENTOS DE LAS PERSONAS
    DE 65 AÑOS O MÁS
    """

    query = """
        SELECT
            dc.NU_CTE_COD AS 'CODIGO CLIENTE',
            dc.EDAD AS 'EDAD CLIENTE',
            dc.CD_ESTADO AS 'CD_ESTADO',
            dc.CD_POSTAL AS 'CD_POSTAL',
            dc.CD_SEXO AS 'CD_SEXO',
            tc.FH_CORTE AS 'MES TRANSACCION',
            tc.FH_OPERACION AS 'DIA TRANSACCION',
            cg.NB_GIRO AS 'GIRO COMERCIAL',
            cg.NB_SUBGIRO AS 'SUBGIRO'
        FROM
            descripcion_cliente AS dc
        RIGHT JOIN
            transacciones_cliente AS tc
        ON
            tc.NU_CTE_COD = dc.NU_CTE_COD
        RIGHT JOIN
            catalogo_giros AS cg
        ON
            tc.NU_AFILIACION = cg.NU_AFILIACION
        WHERE
            dc.EDAD >= 65;
    """

    connector.execute(query)

    movimientos = connector.fetchall()
    movimientos = [movimiento[0:] for movimiento in movimientos]

    return movimientos

obtener_movimientos()

[('1VLE6Y9G',
  '79.0',
  'CH',
  '32390.0',
  'M',
  '01/08/2022',
  '30/08/2022',
  'GRANDES SUPERFICIES',
  'TIENDAS'),
 ('1VLE6Y9G',
  '79.0',
  'CH',
  '32390.0',
  'M',
  '01/09/2022',
  '22/09/2022',
  'GRANDES SUPERFICIES',
  'AUTOSERVICIOS'),
 ('1VLE6Y9G',
  '79.0',
  'CH',
  '32390.0',
  'M',
  '01/07/2022',
  '15/07/2022',
  'GRANDES SUPERFICIES',
  'AUTOSERVICIOS'),
 ('1VLE6Y9G',
  '79.0',
  'CH',
  '32390.0',
  'M',
  '01/08/2022',
  '25/08/2022',
  'VENTAS AL DETALLE (RETAIL)',
  'COMPUTADORAS   EQUIPO PERIFERICO DE COMPUTADORAS, SOFTWARE'),
 ('1VLE6Y9G',
  '79.0',
  'CH',
  '32390.0',
  'M',
  '01/05/2022',
  '01/05/2022',
  'VENTAS AL DETALLE (RETAIL)',
  'COMPUTADORAS   EQUIPO PERIFERICO DE COMPUTADORAS, SOFTWARE'),
 ('1VLE6Y9G',
  '79.0',
  'CH',
  '32390.0',
  'M',
  '01/09/2022',
  '27/09/2022',
  'VENTAS AL DETALLE (RETAIL)',
  'COMPUTADORAS   EQUIPO PERIFERICO DE COMPUTADORAS, SOFTWARE'),
 ('1VLE6Y9G',
  '79.0',
  'CH',
  '32390.0',
  'M',
  '01/07/2022',
  '25/07